# Каналы

Горутины для взаимодействия между собой пользуются **каналами**(channel). С помощью канала одна горутина может отправить какие-либо значения другой горутине. Каждый канал поддерживает определённый тип данных, который называется **типом элементов** канала(channel's element type). Тип канала, элементы которого имеют тип int, можно записать как "chan int".

Создать канал можно встроенной функцией make:

In [1]:
ch := make(chan int)

Как и в случае с словарями, канал является **ссылкой** на структуру данных, которую создаёт функция make. При копировании канала или передаче в функцию происходит копирования ссылки, т.е. функция ссылается на тот же канал, что и вызывающая сторона. Как и для других ссылочных типов **нулевым значением** для канала является nil.

Два канала одинакового типа можно сравнивать между собой, используя ==. Каналы считаются **равными**, если они ссылаются на одну и ту же структуру данных. Канал также можно сравнивать с nil.

Каналы поддерживают две основополагающие операции: **отправка**(send) и **получение**(receive). Вместе эти две операции называют взаимодействиями(communications). Операция отправки посылает значение из одной горутины в другую, которая извлекает это значение с помощью операции получения. Обе эти операции записываются с помощью **<-**. При отправке <- отделяёт канал от значения, при получении <- стоит слева от канала:

In [ ]:
ch <- x  // Отправка

x = <-ch // Получение значения из канала и его присваивание переменной.
<-ch     // Просто извлечение значения из канала.

Каналы также поддерживают ещё одну операцию: **закрытие**(close). Закрытие устанавливает флаг, который говорит, что по этому каналу нельзя больше отправлять данные: если это сделать, то будет паника. Операция получения будет извлекать из закрытого канала данные, которые в него были отправлены до закрытия, пока в нём не останется ничего. После опустошения канала операция получения будет давать нулевые значения для типа элементов канала. Закрыть канал можно вызовом встроенной функции close:

In [ ]:
close(ch)

Из одного канала могут извлекать данные несколько горутин, также же как и несколько горутин могут отправлять значения в него.

Канал, который создаётся простым вызовом make, называется **небуферизированным**(unbuffered) каналом, но make поддерживает и необъязательный второй целочисленный аргумент, который называют **вместительностью**(capacity) канала. Если вместительность не равна 0, то создаётся **буферизированный** канал:

In [ ]:
ch = make(chan int)    // Небуферизированный канал.
ch = make(chan int, 0) // Небуферизированный канал.
ch = make(chan int, 3) // Буферизированный канал вместительности 3.

Далее мы более подробно рассмотрим эти два вида каналов.

## Небуферизированные каналы

Операция отправки для небуферизированного канала **блокирует** горутину пока другая горутина не выполнит операцию получения для этого канала. В этот момент происходит передача значения и обе горутины могут продолжить выполнение. И наоборот, если сначала была выполнена операция получения, то получащая горутина будет заблокирована, пока другая горутина не отправить значение  в этот же канал.

Взаимодействие с помощью небуферизированного канала приводит к **синхронизации** отправляющей и получающей горутин. Поэтому небуферизированные каналы иногда называют синхронными. При отправке значения в небуферизированный канал, получение значения происходит **до того** как отправляющая горутина выйдет из блокировки.

При обсуждении конкурентности когда мы говорим, что x случается до y, мы не только имеем в виду, что x произошло раньше во времени чем y, но и то, что такое поведение гарантировано. Когда x случается ни раньше и не позже чем y, мы говорим, что x **конкурентно с** y. Это необъязательно означает, что x и y произошли одновременно. Это означает, что мы не можем ничего предполагать о порядке происхождения этих событий.

Иногда можно увидеть код, в котором создаются каналы типом элементов которого является пустая структура struct{}:

In [ ]:
ch := make(chan struct{})

Такой канал используется для того, чтобы уведомить получающую горутину, что произошло событие, а само значение, полученно из канала, не играет никакой роли.

Пример:

In [ ]:
package main

import (
	"fmt"
	"time"
)

func main() {
	// Конвейер из трёх горутин.

	naturals := make(chan int)
	squares := make(chan int)

	// Отправляем в канал naturals натуральные числа с задержкой в 1 сек.
	go func() {
		n := 1
		for {
			naturals <- n
			n++
			time.Sleep(1 * time.Second)
		}
	}()

	// Извлекаем значения из канала naturals, находим их квадрат и отправляем дальше в канал squares.
	go func() {
		for {
			n := <-naturals
			squares <- n * n
		}
	}()

	// Выводим значения из канала squares(основная горутина).
	for {
		fmt.Println(<-squares)
	}
}

Этот пример выводит бесконечно квадраты натуральных чисел. Но что делать, если нам нужно было вывести только ограниченное количество квадратов? Для этого нужно закрыть канал. Любые операции отправки в закрытый канал будут приводить к панике. После того как закрытый канал был полностью **высушен**(drained), т.е. из него будут извлечены все значения, последующие операции получения будут без блокировки сразу давать нулевые значения для типа элементов канала. Нет специально выделенной функции или способа, с помощью которого можно узнать закрыт ли канал. Но есть **другой вид** операции получения из канала, который даёт два значения: извлечённое из канала значения и булев флаг, который равен true при удачном извлечении значения из канала и false, если операция получения была выполнена над закрытым и высушенным каналом. С учётом всего сказанного горутину для возведения в квадрат можно было бы переписать так:

In [ ]:
go func() {
    for {
        n, ok := <- naturals
        if !ok {
            break // Канал был закрыт и высушен.
        }
        squares <- n * n
    }
    close(squares)
}()

Синтаксис ручной проверки закрытия и осушения канала является немного неуклюжим. Поэтому для получения всех значений из канала можно воспользоваться циклом for, который **автоматически** завершится, когда в закрытом канале ничего не останется. Т.е. код выше можно переписать в таком виде:

In [ ]:
go func() {
    for {
        for n := range naturals {
            squares <- n * n
        }
    }
    close(squares)
}()

В окончательном виде программу можно переписать так:

In [ ]:
package main

import (
	"fmt"
	"time"
)

func main() {
	// Конвейер из трёх горутин.

	naturals := make(chan int)
	squares := make(chan int)

	// Отправляем в канал naturals натуральные числа от 1 до 3 включительно с задержкой в 1 сек.
	go func() {
		for n := 1; n <= 3; n++ {
			naturals <- n
			time.Sleep(1 * time.Second)
		}
		close(naturals)
	}()

	// Извлекаем значения из канала naturals, находим их квадрат и отправляем дальше в канал squares.
	go func() {
		// Цикл завершится, когда канал naturals будет закрыт и полностью высушен.
		for n := range naturals {
			squares <- n * n
		}
		close(squares)
	}()

	// Выводим значения из канала squares(основная горутина).
	for n := range squares {
		fmt.Println(n)
	}
}

**Необъязательно** нужно закрывать канал, который больше не будет использоваться. Если сборщик мусора обнаружит, что канал является недостижимым, то он освободит ресурсы, выделенные для этого канала, даже если он не был закрыт. Закрывать нужно лишь в том случае, когда мы явно хотим сообщить, что данные закончились.

### Однонаправленные каналы

По мере роста программы естественным образом происходит разбиение больших функций на более мелкие. В нашем предыдущем примере программа состояла из трёх горутин, которые взаимодействовали по двум каналам, которые были локальными переменными функции main. Эту программу можно было бы разбить на три функции:

In [ ]:
func counter(out chan int)
func squarer(in chan int, out chan int)
func printer(in chan int)

Хотя название аргументов функций и намекают на предполагаемый способ использования каждого канала, ничто не запрещает функции squarer, например, отправить значение в канал in, хотя логически одна должна только получать из него значения. Для того чтобы **задокументировать и предотвратить** неправильное использование канала система типов Go поддерживает **однонаправленные каналы**, в которые можно только отправлять значения или только получать. Тип **chan<- int** это тип канала, в который можно только отправлять, а тип **<-chan int** это тип канала, из которого можно только что-то получать. Нарушение предполагаемого способа работы с каналом будут обнаружены на этапе компиляции. Также ошибкой компиляции будет попытка закрытия канала, из которого можно только получать значения.

С учётом однонаправленных каналов, наш пример можно переписать так:

In [ ]:
package main

import (
	"fmt"
	"time"
)

func counter(out chan<- int) {
	for n := 1; n <= 3; n++ {
		out <- n
		time.Sleep(1 * time.Second)
	}
	close(out)
}

func squarer(in <-chan int, out chan<- int) {
	for n := range in {
		out <- n * n
	}
	close(out)
}

func printer(in <-chan int) {
	for n := range in {
		fmt.Println(n)
	}
}

func main() {
	naturals := make(chan int)
	squares := make(chan int)

	go counter(naturals)
	go squarer(naturals, squares)
	printer(squares)
}

## Буферизированные каналы

Буферизированный канал можно рассматривать как очередь элементов. Максимальный размер очереди определяется при создании канала в функции make через аргумент вместительности(capacity). Пример:

In [ ]:
// Буферизированный канал, который может хранить максимум 3 строки.
ch := make(chan string, 3)

Операция отправки добавляет элемент в конец очереди, а операция получения извлекает элемент с её начала. Если очередь полностью заполнена, то операция отправки блокируется до момента пока какая-либо другая горутина не извлечёт значение из очереди. И наоборот, операция получения блокируется, если очередь пустая пока значения не будут отправлены в неё. В созданный нами канал можно отправлять до трёх значений без блокировки:

In [ ]:
ch <- "A"
ch <- "B"
ch <- "C"

Если попытаться ещё раз отправить значение в канал, то оно приведёт к блокировке. Если мы извлечём значение из канала, то он будет не пустым и не полностью заполненным. Тогда операция отправки или получения не приведёт к блокировке, т.е. эти операции могут быть выполнены без синхронизации между собой как это было в случае с небуферизированными каналами.

Встроенная функция **cap** для каналов возвращает его вместительность, а **len** - текущий размер очереди значений.